<a href="https://colab.research.google.com/github/SABR007/NLP_Projects/blob/main/Copy_of_Exercise_2_Word_Embeddings_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

### Source: [link](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
PATH = '/content/drive/My Drive/Colab Notebooks/NLP/'

Mounted at /content/drive


In [ ]:
# Author: Robert Guthrie

import torch
assert torch.cuda.is_available(), 'GPU not available'
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

In [ ]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU is not available or not being used. Using CPU instead.")

GPU is available and being used.


In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss() # Negative Log Likelihood Loss
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print("Loss in Epoch {ep}: {l}".format(ep=epoch, l=np.round(total_loss, 2))) # The loss decreased every iteration over the training data!
    losses.append(total_loss)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
Loss in Epoch 0: 521.58
Loss in Epoch 1: 518.9
Loss in Epoch 2: 516.24
Loss in Epoch 3: 513.6
Loss in Epoch 4: 510.98
Loss in Epoch 5: 508.38
Loss in Epoch 6: 505.79
Loss in Epoch 7: 503.22
Loss in Epoch 8: 500.67
Loss in Epoch 9: 498.12


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.


## Exercise Layout
### 1. <u>Training CBOW Embeddings</u>
1.1) Implement a CBOW Model by completing ```class CBOW(nn.Module)``` and train it on ```raw_text```.    

1.2) Load Datasets ```tripadvisor_hotel_reviews_reduced.csv``` and ```scifi_reduced.txt```.     

1.3) Decide preprocessing steps by completing the function ```def custom_preprocess()```. Describe your decisions. Note that it's your choice to create different preprocessing functions for hotel reviews and scifi datasets or use the same preprocessing function.             

1.4) Train CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.   

1.5) Train CBOW5 with a context width of 5 (in both directions) for the Hotel Reviews dataset. Are predictions made by the model sensitive towards the context size?
     
1.6) Train CBOW2 with a context width of 2 (in both directions) for the Sci-Fi story dataset.  


### 2. <u>Test your Embeddings</u>
Note - Do the following for CBOW2, and optionally for CBOW5

2.1) For the hotel reviews dataset, choose 3 nouns, 3 verbs, and 3 adjectives. Make sure that some nouns/verbs/adjectives occur frequently in the corpus and that others are rare. For each of the 9 chosen words, retrieve the 5 closest words according to your trained CBOW2 model. List them in your report and comment on the performance of your model: do the neighbours the model provides make sense? Discuss.   

2.2) Do the same for Sci-Fi dataset.   

2.3) How does the quality of the hotel review-based embeddings compare with the Sci-fi-based embeddings? Elaborate.   

2.4) Choose 2 words and retrieve their 5 closest neighbours according to hotel review-based embeddings and the Sci-fi-based embeddings. Do they have different neighbours? If yes, can you reason why?    

2.5) What are the differences between CBOW2 and CBOW5 ? Can you "describe" them?   


### Tips

1. Switch from CPU to a GPU instance after you have confirmed that your training procedure is working correctly.
2. You can always save your intermediate results (embeddings, preprocessed dataset, model, etc.) in your google drive via colab



### 1.1 Create a CBOW Model by completing ```class CBOW(nn.Module)``` and test it on ```raw_text```
Implement CBOW in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.

In [ ]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        #self.linear2 = nn.Linear(128, vocab_size)
        self.activation_fn = nn.LogSoftmax(dim=-1)


    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        #embeds = embeds.view(1, -1)
        embeds = torch.mean(embeds, dim=0).view(1, -1)

        z = self.linear1(embeds)
        out = self.activation_fn(z)
        return out

    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long)
        return self.embeddings(word).view(1, -1)
        """embeds = torch.sum(embeds, dim=0)
        out = self.linear1(embeds)
        out = self.activation_fn(out)
        return out
        pass"""

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([27, 36, 47,  1])

In [ ]:
model =  CBOW(vocab_size, embedding_dim = 100)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
model.to(device)


for epoch in range(10):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)
        context_vector = context_vector.to(device)
        out = model(context_vector)

        loss = loss_function(out, torch.tensor([word_to_ix[target]], dtype=torch.long, device = device) )
        total_loss += loss.item()

        #model.zero_grad()
        #optimizer.zero_grad()
        #loss = loss_function(out, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

    print("Loss in Epoch {ep}: {l}".format(ep=epoch, l=np.round(total_loss, 2))) # The loss decreased every iteration over the training data!
    #

Loss in Epoch 0: 227.69
Loss in Epoch 1: 147.22
Loss in Epoch 2: 55.62
Loss in Epoch 3: 11.9
Loss in Epoch 4: 3.06
Loss in Epoch 5: 1.47
Loss in Epoch 6: 0.39
Loss in Epoch 7: 0.07
Loss in Epoch 8: 0.02
Loss in Epoch 9: 0.01


### 1.2 Load Datasets

In [ ]:
### Load Datasets tripadvisor_hotel_reviews_reduced.csv and scifi_reduced.txt

!gdown 1foE1JuZJeu5E_4qVge9kExzhvF32teuF # For Hotel Reviews
!gdown 13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75 # For Scifi-Text

Downloading...
From: https://drive.google.com/uc?id=1foE1JuZJeu5E_4qVge9kExzhvF32teuF
To: /content/tripadvisor_hotel_reviews_reduced.csv
100% 7.36M/7.36M [00:00<00:00, 30.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75
To: /content/scifi_reduced.txt
100% 43.1M/43.1M [00:00<00:00, 93.4MB/s]


In [ ]:
#open("tripadvisor_hotel_reviews_reduced.csv", "r").read()

### 1.3 Preprocess Datasets
### 🗒❓ Describe your decisions for preprocessing the datasets

In [ ]:
### Complete the preprocessing function and apply it to the datasets

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def custom_preprocess(text, is_sci_fi=False):
    text = text.lower()
    text = "".join([char for char in text if char.isalnum() or char.isspace()]) #remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    if is_sci_fi:
        # Add custom sci-fi stop words
        stop_words.update(["spaceship", "planet", "alien"])
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens



hotel_reviews = open("tripadvisor_hotel_reviews_reduced.csv", "r").read()
sci_fi_story = open("scifi_reduced.txt", "r").read()

hotel_reviews_processed = custom_preprocess(hotel_reviews)
sci_fi_story_processed = custom_preprocess(sci_fi_story, is_sci_fi=True)

print(len(hotel_reviews_processed))
print(len(sci_fi_story_processed))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1023892
3954325


In [52]:
vocab1 = set(hotel_reviews_processed)
#vocab2 = set(sci_fi_story_processed
vocab_size1 = len(vocab1)

word_to_ix = {word: i for i, word in enumerate(vocab1)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

data = []
for i in range(2, len(hotel_reviews_processed) - 2):
    context1 = [hotel_reviews_processed[i - 2], hotel_reviews_processed[i - 1],
               hotel_reviews_processed[i + 1], hotel_reviews_processed[i + 2]]
    target = hotel_reviews_processed[i]
    data.append((context1, target))
print(data[:5])

#print(data)


[(['reviewrating', 'fantastic', 'large', 'hotel'], 'service'), (['fantastic', 'service', 'hotel', 'caters'], 'large'), (['service', 'large', 'caters', 'business'], 'hotel'), (['large', 'hotel', 'business', 'corporates'], 'caters'), (['hotel', 'caters', 'corporates', 'serve'], 'business')]


### 1.4 Train CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.

In [57]:
'''class CBOW2(nn.Module):
    def __init__(self, vocab_size1, embedding_dim):
        super(CBOW2, self).__init__()

        self.embeddings = nn.Embedding(vocab_size1, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, vocab_size1)
        #self.linear2 = nn.Linear(128, vocab_size)
        self.activation_fn = nn.LogSoftmax(dim=-1)


    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        #embeds = embeds.view(1, -1)
        embeds = torch.mean(embeds, dim=0).view(1, -1)

        z = self.linear1(embeds)
        out = self.activation_fn(z)
        return out

    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long)
        return self.embeddings(word).view(1, -1)


# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(data, word_to_ix):
    idxs = [word_to_ix[w] for w in data]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example'''



import torch.nn as nn
import torch

class CBOW2(nn.Module):
    def __init__(self, vocab_size1, embedding_dim, context_size, dropout_prob=0.2):
        super(CBOW2, self).__init__()
        self.embeddings = nn.Embedding(vocab_size1, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size1)
        self.activation_fn = nn.LogSoftmax(dim=-1)
        self.dropout = nn.Dropout(dropout_prob) #add dropout
        self.context_size = context_size


    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.mean(embeds, dim=0).view(1, -1)
        embeds = self.dropout(embeds) #apply dropout
        z = self.linear1(embeds)
        out = self.activation_fn(z)
        return out

    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long)
        return self.embeddings(word).view(1, -1)


def make_context_vector(data, word_to_ix, context_size): #add context_size
    #Handle edge cases for context window exceeding sentence boundaries
    center_index = len(data) // 2
    start_index = max(0, center_index - context_size)
    end_index = min(len(data), center_index + context_size + 1)
    idxs = [word_to_ix[w] for w in data[start_index:end_index] if w in word_to_ix]
    return torch.tensor(idxs, dtype=torch.long)

GPU is available and being used.


In [ ]:
#model.to(device)

In [62]:
model = CBOW2(vocab_size1, embedding_dim=10, context_size=2) # Added context_size
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (remains the same as the corrected version from the previous response)
for epoch in range(10):
    total_loss = 0
    for context, target in data:
        optimizer.zero_grad()
        context_vector = make_context_vector(context, word_to_ix, 2) # Pass context_size to make_context_vector
        context_vector = context_vector.to(device)
        out = model(context_vector)
        log_probs = F.log_softmax(out, dim=1)
        target_index = torch.tensor([word_to_ix[target]], dtype=torch.long, device=device)
        loss = loss_func(log_probs, target_index)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(data)
    print(f"Loss in Epoch {epoch}: {np.round(avg_loss, 2)}")


Loss in Epoch 0: 9.8


KeyboardInterrupt: 

In [54]:
"""model = CBOW2(vocab_size1, embedding_dim=10)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

loss_func = torch.nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)
print(context1)
#total_loss = 0  # Initialize total loss for each epoch
for epoch in range(10):
    total_loss = 0 # Reset total loss for each epoch
    optimizer.zero_grad()
    #print(context1)  #Important: zero gradients before each iteration
    context_vector = make_context_vector(context1, word_to_ix)
    context_vector = context_vector.to(device)
    out = model(context_vector)
    # Apply log_softmax before NLLLoss
    log_probs = F.log_softmax(out, dim=1) # dim=1 for classification across classes
    target_index = torch.tensor([word_to_ix[target]], dtype=torch.long, device=device)
    print(f"Target index: {target_index}") #debug statement
    loss = loss_func(log_probs, target_index)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
    print(f"Loss in Epoch {epoch}: {np.round(total_loss, 2)}")
"""

['amsterdam', 'carry', 'problem', '5']


TypeError: unhashable type: 'list'

### 1.5 Train CBOW5 with a context width of 5 (in both directions) for the Hotel Reviews dataset.  

🗒❓ Are predictions made by the model sensitive towards the context size?

In [1]:
model = CBOW2(vocab_size1, embedding_dim=10, context_size=5) # Added context_size
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (remains the same as the corrected version from the previous response)
for epoch in range(10):
    total_loss = 0
    for context, target in data:
        optimizer.zero_grad()
        context_vector = make_context_vector(context1, word_to_ix, 2) # Pass context_size to make_context_vector
        context_vector = context_vector.to(device)
        out = model(context_vector)
        log_probs = F.log_softmax(out, dim=1)
        target_index = torch.tensor([word_to_ix[target]], dtype=torch.long, device=device)
        loss = loss_func(log_probs, target_index)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(data)
    print(f"Loss in Epoch {epoch}: {np.round(avg_loss, 2)}")


NameError: name 'CBOW2' is not defined

### 1.6 Train CBOW2 with a context width of 2 (in both directions) for the Sci-Fi story dataset

In [ ]:
#vocab1 = set(hotel_reviews_processed)
vocab2 = set(sci_fi_story_processed)
vocab_size2 = len(vocab2)

word_to_ix = {word: i for i, word in enumerate(vocab2)}

#ix_to_word = {ix: word for i, word in enumerate(vocab)}

data2 = []
for i in range(2, len(sci_fi_story_processed) - 2):
    context2 = [sci_fi_story_processed[i - 2], sci_fi_story_processed[i - 1],
               sci_fi_story_processed[i + 1], sci_fi_story_processed[i + 2]]
    target2 = sci_fi_story_processed[i]
    data2.append((context2, target2))
print(data2[:5])

#print(data)


In [ ]:

class CBOW2(nn.Module):
    def __init__(self, vocab_size1, embedding_dim, context_size, dropout_prob=0.2):
        super(CBOW2, self).__init__()
        self.embeddings = nn.Embedding(vocab_size2, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size1)
        self.activation_fn = nn.LogSoftmax(dim=-1)
        self.dropout = nn.Dropout(dropout_prob) #add dropout
        self.context_size = context_size


    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.mean(embeds, dim=0).view(1, -1)
        embeds = self.dropout(embeds) #apply dropout
        z = self.linear1(embeds)
        out = self.activation_fn(z)
        return out

    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]], dtype=torch.long)
        return self.embeddings(word).view(1, -1)


def make_context_vector(data, word_to_ix, context_size): #add context_size
    #Handle edge cases for context window exceeding sentence boundaries
    center_index = len(data2) // 2
    start_index = max(0, center_index - context_size)
    end_index = min(len(data2), center_index + context_size + 1)
    idxs = [word_to_ix[w] for w in data2[start_index:end_index] if w in word_to_ix]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
model = CBOW2(vocab_size2, embedding_dim=10, context_size=2) # Added context_size
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

loss_func = torch.nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (remains the same as the corrected version from the previous response)
for epoch in range(10):
    total_loss = 0
    for context, target in data2:
        optimizer.zero_grad()
        context_vector = make_context_vector(context2, word_to_ix, 2) # Pass context_size to make_context_vector
        context_vector = context_vector.to(device)
        out = model(context_vector)
        log_probs = F.log_softmax(out, dim=1)
        target_index = torch.tensor([word_to_ix[target2]], dtype=torch.long, device=device)
        loss = loss_func(log_probs, target_index)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(data2)
    print(f"Loss in Epoch {epoch}: {np.round(avg_loss, 2)}")


### 2.1 For the hotel reviews dataset, choose 3 nouns, 3 verbs, and 3 adjectives. (CBOW2 and optionally for CBOW5)
Make sure that some nouns/verbs/adjectives occur frequently in the corpus and that others are rare. For each of the 9 chosen words, retrieve the 5 closest words according to your trained CBOW2 model.    

🗒❓ List them in your report (at the end of this notebook) and comment on the performance of your model: do the neighbours the model provides make sense? Discuss.   


### 2.2 Repeat 2.1 for SciFi Dataset

🗒❓ List your findings for SciFi Dataset as well, similarly to 2.1

### 2.3 🗒❓ How does the quality of the hotel review-based embeddings compare with the Sci-fi-based embeddings? Elaborate.

### 2.4 Choose 2 words and retrieve their 5 closest neighbours according to hotel review-based embeddings and the Sci-fi-based embeddings.

🗒❓ Do they have different neighbours? If yes, can you reason why?

### 2.5 🗒❓ What are the differences between CBOW2 and CBOW5 ? Can you "describe" them?    

### Report
The lab report should contain a detailed description of the approaches you have used to solve this exercise. Please also include results.

Answers for the questions marked 🗒❓ goes here as well